## Import required libraries

In [13]:
# import all libraries and modules
import sys; import os
import pytraj as pt
from sklearn.decomposition import PCA
import numpy as np
import scipy

# for plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import art3d

## Import modules from BKit

In [21]:
from BKit.Utils import SplitEvenOdd, PlotSelected, pathpatch_2d_to_3d
from BKit.ConstructMilestones3D import SortFrames, rotation_matrix


## Set model parameters

In [15]:
ml_length = 0.8    # distance between milestones
dr = 2.0           # milestone radius (disk radius)
dim = 3            # pc dimention    
yz_pad = 8
dz = 0.2

## Set  path & load previoulsly saved data

In [37]:
DIR_SAVE = '../output'
complex_name = 'Cyc_asp'
inp = np.load(DIR_SAVE + '/MlPosNorms.npy')
dat = np.load('/home/talant/Projects/UCRiver/Milestoning/ForRuben/output/PCA' + '/PCA.npy')
pathAll = inp[:,0:3];  normalsAll = inp[:,3:6]

In [17]:
diskID, midID = SplitEvenOdd(N=pathAll.shape[0])
pathMid = pathAll[midID];       pathP = pathAll[diskID]    
normalsMid = normalsAll[midID]; normals = normalsAll[diskID]       

n_cells = normalsMid.shape[0]
n_disks = normals.shape[0]

In [18]:
SortF = SortFrames(dat,dr,dz)
datMid, CellIndx = SortF.SortAllPoints(normals, normalsMid, pathP, pathMid, SortMethod='middle')

In [19]:
nf = dat.shape[0] 
n_sel = datMid.shape[0]
print('total frames',nf)
print('Number of selected points', n_sel)
print('Number of Cells ', n_cells)
print('Number of Disks ', n_disks)

total frames 8620000
Number of selected points 2500699
Number of Cells  61
Number of Disks  62


In [61]:
#tmp=datMid[(datMid[:,3] > 0) & (datMid[:,3] < 18)]
#n_sel = tmp.shape[0]

rand_sel = np.random.randint(0, n_sel, 50000)
datPlot = datMid[rand_sel,:]

## Label all States & Visualize Subset and Save

In [68]:
%matplotlib widget
plotOrig = True
dat_sel = datPlot
fs=9

#n_disks = normals.shape[0]
#fig = plt.figure(figsize = [4.2, 3.4])
fig = plt.figure(figsize = [7.2, 5.4])
ax = plt.axes(projection='3d')
  
#plot path poits
ax.plot3D(pathP[:,0], pathP[:,1], pathP[:,2], c='black', marker='o', markersize=2, alpha=0.8)
ax.set_xlabel('PC1', fontsize=fs)
ax.set_ylabel('PC2',fontsize=fs)
ax.set_zlabel('PC3',fontsize=fs)

#plotting disks
for i in range(n_disks):
    c = Circle((0,0), dr, facecolor='grey', alpha=0.4)
    ax.add_patch(c)
    pathpatch_2d_to_3d(c, pathP[i], normal = normals[i])
    #ax.text(pathP[i,0] +0.1*dr, pathP[i,1] +0.1*dr ,
    #        pathP[i,2] +0.1*dr , str(i), 'x', size=fs)
    #ax.text(pathP[i,0] - 1.2*dr, pathP[i,1] + 0.4*dr ,
    #        pathP[i,2] - 2.3*dr, str(i), 'x', size=fs)

#plot selected points
p = ax.scatter3D(dat_sel[:,0], dat_sel[:,1], dat_sel[:,2], c = dat_sel[:,3],
                 alpha=.40, s = 2., cmap = 'prism')
    
#p = ax.scatter3D(datOut[:,0], datOut[:,1], datOut[:,2], c = 'grey', s=2., alpha=0.4)
    
x_min, y_min, z_min = pathP.min(axis=0) 
x_max, y_max, z_max = pathP.max(axis=0)
ax.set_xlim3d(x_min, x_max)
ax.set_ylim3d(y_min - yz_pad, y_max + yz_pad)
ax.set_zlim3d(z_min - yz_pad, z_max + yz_pad)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.set_xlim3d(-5,25); ax.set_ylim3d(-5,20); ax.set_zlim3d(-10,0)
#ax.set_axis_off()
fig.tight_layout()
plt.savefig(DIR_SAVE + complex_name + 'states.png', dpi=600)
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
#ini=20000*100; fin=ini + 999

In [56]:
#tmp_dat = dat[ini: fin]

In [62]:
%matplotlib widget
plotOrig = True
dat_sel = datPlot
fs = 9

#n_disks = normals.shape[0]
fig = plt.figure(figsize = [6.2, 4.4])
ax = plt.axes(projection='3d')
  
#plot path poits
ax.plot3D(pathP[:,0], pathP[:,1], pathP[:,2], c='black', marker='o', markersize=2, alpha=0.8)
#ax.plot3D(tmp_dat[:,0], tmp_dat[:,1], tmp_dat[:,2], c='black')#, marker='-', markersize=2, alpha=1.0)
ax.set_xlabel('PC1', fontsize=fs)
ax.set_ylabel('PC2',fontsize=fs)
ax.set_zlabel('PC3',fontsize=fs)

#plotting disks
for i in range(n_disks):
    c = Circle((0,0), dr, facecolor='grey', alpha=0.4)
    ax.add_patch(c)
    pathpatch_2d_to_3d(c, pathP[i], normal = normals[i])
    ax.text(pathP[i,0] +0.1*dr, pathP[i,1] +0.1*dr ,
            pathP[i,2] +0.1*dr , str(i), 'x', size=fs)
    #ax.text(pathP[i,0] - 1.2*dr, pathP[i,1] + 0.4*dr ,
    #        pathP[i,2] - 2.3*dr, str(i), 'x', size=fs)

#plot selected points
p = ax.scatter3D(dat_sel[:,0], dat_sel[:,1], dat_sel[:,2], c = dat_sel[:,3],
                 alpha=.10, s = 2., cmap = 'prism')
    
#p = ax.scatter3D(datOut[:,0], datOut[:,1], datOut[:,2], c = 'grey', s=2., alpha=0.4)
    
x_min, y_min, z_min = pathP.min(axis=0) 
x_max, y_max, z_max = pathP.max(axis=0)
ax.set_xlim3d(x_min, x_max)
ax.set_ylim3d(y_min - yz_pad, y_max + yz_pad)
ax.set_zlim3d(z_min - yz_pad, z_max + yz_pad)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
#ax.set_xlim3d(0,160); ax.set_ylim3d(-50,70); ax.set_zlim3d(-40,60)
ax.set_axis_off()
fig.tight_layout()
#plt.savefig(DIR_SAVE + complex_name + 'states.png', dpi=600)
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#%matplotlib widget
#PlotSelected(datPlot, dr, pathP, normals, yz_pad, figsize=[8,6])



In [8]:
print("writing outputs to " + DIR_SAVE)
np.save(DIR_SAVE + '/CellIndx.npy', CellIndx)

writing outputs to ../output
